In [ ]:
!git clone https://github.com/MockaWolke/featout_hackathon
%cd featout_hackathon
!bash load_data.sh
!pip install -e .
!pip install captum

Cloning into 'featout_hackathon'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 145 (delta 20), reused 44 (delta 14), pack-reused 91
Receiving objects: 100% (145/145), 32.43 MiB | 18.08 MiB/s, done.
Resolving deltas: 100% (61/61), done.
/content/featout_hackathon
--2023-09-14 16:55:05--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.6.204.160, 2600:1407:3c00:a86::317f, 2600:1407:3c00:a93::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.6.204.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   133MB/s    in 5.8s    

2023-09-14 16:55:11 (136 MB/s) - ‘kagglecatsanddogs_534

In [ ]:
import os
import time
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import csv
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from featout_exp.utils import load_data, load_model, load_data_raw_dataset
from featout_exp.ours import *

In [ ]:
BATCHSIZE = 32
NUM_EPOCHS = 10
EXP_NAME = "Ours_Featout"
TESTING = False
BLUR_METHOD = blur_featurs
# algorithm to derive the model's attention
ATTENTION_ALGORITHM = simple_gradient_saliency

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = load_model(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train_dataset = load_data_raw_dataset("train")

train_featout = Featout(train_dataset, "test_images", do_plotting= False, device= device)

train_loader = torch.utils.data.DataLoader(
    train_featout, batch_size= 32, shuffle=True, num_workers=0
)


test_loader = load_data("test", 32, False)


EXP_PATH = os.path.join("logs",EXP_NAME)

os.makedirs(EXP_PATH, exist_ok=True)
# Initialize TensorBoard writer
writer = SummaryWriter(EXP_PATH)

# Prepare a CSV file for logging
csv_file = open(os.path.join(EXP_PATH, "logs.csv"), "w")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Test Loss", "Test Accuracy"])

57

In [ ]:
def exploration_rate(epoch):
    exp_rate = epoch / NUM_EPOCHS
    if exp_rate > 1:
        return 1
    return exp_rate

In [ ]:
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    corrects = 0
    total = 0

    bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    if epoch > 0:
        train_loader.dataset.start_featout(
            model,
            blur_method=BLUR_METHOD,
            algorithm=ATTENTION_ALGORITHM,
            exp_rate=exploration_rate(epoch),
            max_clusters=10
        )

    for step, (inputs, labels) in enumerate(bar):


        if TESTING == True and step > 2:
            break

        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        corrects += torch.sum(preds == labels.data)
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = corrects.double() / total

    # Evaluate on test data
    model.eval()
    test_running_loss = 0.0
    test_corrects = 0
    test_total = 0

    with torch.no_grad():
        for step,  (inputs, labels) in enumerate(tqdm( test_loader)):
            if TESTING == True and step > 2:
                 break

            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * inputs.size(0)
            test_corrects += torch.sum(preds == labels.data)
            test_total += labels.size(0)

    test_loss = test_running_loss / test_total
    test_acc = test_corrects.double() / test_total

    # Log metrics to TensorBoard
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Accuracy/train", train_acc, epoch)
    writer.add_scalar("Loss/test", test_loss, epoch)
    writer.add_scalar("Accuracy/test", test_acc, epoch)

    # Log metrics to CSV
    csv_writer.writerow([epoch + 1, train_loss, train_acc.item(), test_loss, test_acc.item()])
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f} - Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")


csv_file.close()
writer.close()


  1%|▏         | 2/157 [00:00<00:23,  6.47it/s]/usr/local/lib/python3.10/dist-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 157/157 [00:18<00:00,  8.33it/s]


Epoch 1/10 - Train Loss: 0.4799 Train Acc: 0.7665 - Test Loss: 1.3895 Test Acc: 0.4584


Epoch 2/10:   0%|          | 1/625 [00:00<01:32,  6.73it/s]


 STARTING FEATOUT 
 


Epoch 2/10:   0%|          | 2/625 [00:00<05:48,  1.79it/s]

...


 STARTING FEATOUT 
 


Epoch 2/10:  11%|█         | 66/625 [00:54<06:59,  1.33it/s]


 STARTING FEATOUT 
 


Epoch 2/10:  11%|█         | 67/625 [00:55<06:54,  1.35it/s]


 STARTING FEATOUT 
 


In [ ]:
torch.save(model.state_dict(), os.path.join(EXP_PATH, "model.pth"))

In [ ]:
!zip -r Hers.zip logs/Standard_Featout


In [ ]:
from google.colab import files
files.download('Hers.zip')